In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os
import tarfile
import urllib
import pandas as pd
import urllib.request

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
 
import category_encoders as ce 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import KFold 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LeakyReLU, Dense

In [2]:
def load_data():
    csv_path ="./train-set.csv"
    return pd.read_csv(csv_path)

In [3]:
pd.options.display.float_format = '{:,.0f}'.format

df = load_data() # "housing" is a pandas DataFrame object containing all the data
df.head() # Display the top five rows of the DataFrame
#Q1 How many attribuites in the data set? Desribe the attribuites .

,CustomerID,Gender,Married,Age,Graduated,Profession,WorkExperience,SpendingScore,FamilySize,Category,Segmentation
0,462809,Male,No,22,No,Healthcare,1,Low,4,Category 4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3,Category 4,A
2,466315,Female,Yes,67,Yes,Engineer,1,Low,1,Category 6,B
3,461735,Male,Yes,67,Yes,Lawyer,0,High,2,Category 6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6,Category 6,A


In [6]:
df['SpendingScore'].value_counts()

Low        4878
Average    1974
High       1216
Name: SpendingScore, dtype: int64

In [7]:
df['Category'].value_counts()

Category 6    5238
Category 4    1089
Category 3     822
Category 2     422
Category 7     203
Category 1     133
Category 5      85
Name: Category, dtype: int64

In [8]:
df['Segmentation'].value_counts()

D    2268
A    1972
C    1970
B    1858
Name: Segmentation, dtype: int64

In [9]:
dictionary_SpendingScore=[{'col':'SpendingScore','mapping':{'Low':1,'Average':2,'High':3}}]
encoder=ce.OrdinalEncoder(cols='SpendingScore',mapping=dictionary_SpendingScore)

x_train_encoded =encoder.fit_transform(df)

dictionary_Category=[{'col':'Category','mapping':{'Category 1':1,'Category 2':2,'Category 3':3,'Category 4':4,'Category 5':5,'Category 6':6,'Category 7':7}}]
encoder=ce.OrdinalEncoder(cols='Category',mapping=dictionary_Category)

x_train_encoded =encoder.fit_transform(x_train_encoded)

dictionary_Segmentation=[{'col':'Segmentation','mapping':{'A':0,'B':1,'C':2,'D':3}}]
encoder=ce.OrdinalEncoder(cols='Category',mapping=dictionary_Segmentation)

x_train_encoded =encoder.fit_transform(x_train_encoded)

#coding the Category to number
x_train_encoded.Gender = pd.Categorical(x_train_encoded.Gender)
x_train_encoded['Gender'] = x_train_encoded.Gender.cat.codes

x_train_encoded.Married = pd.Categorical(x_train_encoded.Married)
x_train_encoded['Married'] = x_train_encoded.Married.cat.codes

x_train_encoded.Graduated = pd.Categorical(x_train_encoded.Graduated)
x_train_encoded['Graduated'] = x_train_encoded.Graduated.cat.codes

x_train_encoded.Profession = pd.Categorical(x_train_encoded.Profession)
x_train_encoded['Profession'] = x_train_encoded.Profession.cat.codes



x_train_encoded.head() 

,CustomerID,Gender,Married,Age,Graduated,Profession,WorkExperience,SpendingScore,FamilySize,Category,Segmentation
0,462809,1,0,22,0,5,1,1,4,4,3
1,462643,0,1,38,1,2,NaN,2,3,4,0
2,466315,0,1,67,1,2,1,1,1,6,1
3,461735,1,1,67,1,7,0,3,2,6,1
4,462669,0,1,40,1,3,NaN,3,6,6,0


In [10]:

x_train_encoded["WorkExperience"] = pd.to_numeric(x_train_encoded["WorkExperience"])


nan_value = float("NaN")
#x_train_encoded.replace(0, nan_value, inplace=True)
x_train_encoded.dropna(subset = ["FamilySize"], inplace=True)
x_train_encoded.dropna(subset = ["WorkExperience"], inplace=True)
 
x_train_encoded["WorkExperience"] = x_train_encoded["WorkExperience"].astype(int)

X=np.array(x_train_encoded[['Gender','Married','Age','Graduated','Profession','WorkExperience','SpendingScore','FamilySize','Category']])

y=np.array(x_train_encoded['Segmentation'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
print(X_train)
print(X_train.shape)
print(y)
print(y.shape)

[[ 1.  1. 39. ...  2.  2.  1.]
 [ 0.  0. 27. ...  1.  4.  6.]
 [ 0.  1. 41. ...  2.  2.  6.]
 ...
 [ 0.  0. 38. ...  1.  8.  2.]
 [ 1.  1. 40. ...  1.  2.  6.]
 [ 0.  0. 55. ...  1.  1.  6.]]
(4878, 9)
[3 1 1 ... 3 1 1]
(6969,)


In [11]:
y_train=np.reshape(y_train, (y_train.shape[0],1))

In [25]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(X_train.shape[0], input_dim=9, activation='sigmoid'))
model.add(keras.layers.Dense(20, activation="sigmoid")) 
model.add(keras.layers.Dense(4,activation='softmax'))

In [26]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [27]:
history = model.fit(X_train, y_train, epochs=30)

Epoch 1/30
153/153 [==============================] - 0s 2ms/step - loss: 1.3350 - accuracy: 0.3483
Epoch 2/30
153/153 [==============================] - 0s 2ms/step - loss: 1.2807 - accuracy: 0.4053
Epoch 3/30
153/153 [==============================] - 0s 2ms/step - loss: 1.2408 - accuracy: 0.4526
Epoch 4/30
153/153 [==============================] - 0s 2ms/step - loss: 1.2084 - accuracy: 0.4623
Epoch 5/30
153/153 [==============================] - 0s 2ms/step - loss: 1.1891 - accuracy: 0.4695
Epoch 6/30
153/153 [==============================] - 0s 2ms/step - loss: 1.1735 - accuracy: 0.4777
Epoch 7/30
153/153 [==============================] - 0s 2ms/step - loss: 1.1626 - accuracy: 0.4918
Epoch 8/30
153/153 [==============================] - 0s 2ms/step - loss: 1.1581 - accuracy: 0.4875
Epoch 9/30
153/153 [==============================] - 0s 2ms/step - loss: 1.1529 - accuracy: 0.4918
Epoch 10/30
153/153 [==============================] - 0s 2ms/step - loss: 1.1490 - accuracy: 0.4945

In [49]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(X_train.shape[0], input_dim=9, activation='relu'))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(50, activation="relu")) 
model.add(keras.layers.Dense(4,activation='softmax'))

In [50]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [51]:
history = model.fit(X_train, y_train, epochs=100)

Epoch 1/100
153/153 [==============================] - 1s 4ms/step - loss: 1.2866 - accuracy: 0.4067
Epoch 2/100
153/153 [==============================] - 1s 4ms/step - loss: 1.2182 - accuracy: 0.4432
Epoch 3/100
153/153 [==============================] - 1s 4ms/step - loss: 1.1761 - accuracy: 0.4682
Epoch 4/100
153/153 [==============================] - 1s 4ms/step - loss: 1.1673 - accuracy: 0.4783
Epoch 5/100
153/153 [==============================] - 1s 4ms/step - loss: 1.1568 - accuracy: 0.4764
Epoch 6/100
153/153 [==============================] - 1s 4ms/step - loss: 1.1521 - accuracy: 0.4809
Epoch 7/100
153/153 [==============================] - 1s 4ms/step - loss: 1.1389 - accuracy: 0.4902
Epoch 8/100
153/153 [==============================] - 1s 4ms/step - loss: 1.1351 - accuracy: 0.4916
Epoch 9/100
153/153 [==============================] - 1s 4ms/step - loss: 1.1348 - accuracy: 0.4947
Epoch 10/100
153/153 [==============================] - 1s 4ms/step - loss: 1.1258 - accura

In [56]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(X_train.shape[0], input_dim=9, activation='relu'))
model.add(keras.layers.Dense(200, activation="relu" ))
model.add(keras.layers.Dense(100, activation="relu")) 
model.add(keras.layers.Dense(200, activation="relu")) 
model.add(keras.layers.Dense(4,activation='softmax'))

In [57]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [58]:
history = model.fit(X_train, y_train, epochs=1500)

Epoch 1/1500
153/153 [==============================] - 1s 7ms/step - loss: 1.2808 - accuracy: 0.3973
Epoch 2/1500
153/153 [==============================] - 1s 7ms/step - loss: 1.2085 - accuracy: 0.4358
Epoch 3/1500
153/153 [==============================] - 1s 8ms/step - loss: 1.1714 - accuracy: 0.4697
Epoch 4/1500
153/153 [==============================] - 1s 8ms/step - loss: 1.1521 - accuracy: 0.4801
Epoch 5/1500
153/153 [==============================] - 1s 8ms/step - loss: 1.1456 - accuracy: 0.4936
Epoch 6/1500
153/153 [==============================] - 1s 7ms/step - loss: 1.1392 - accuracy: 0.4883
Epoch 7/1500
153/153 [==============================] - 1s 7ms/step - loss: 1.1354 - accuracy: 0.4961
Epoch 8/1500
153/153 [==============================] - 1s 7ms/step - loss: 1.1300 - accuracy: 0.5002
Epoch 9/1500
153/153 [==============================] - 1s 8ms/step - loss: 1.1326 - accuracy: 0.4934
Epoch 10/1500
153/153 [==============================] - 1s 8ms/step - loss: 1.125

153/153 [==============================] - 1s 6ms/step - loss: 0.5072 - accuracy: 0.7843
Epoch 160/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.5068 - accuracy: 0.7798
Epoch 161/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.5213 - accuracy: 0.7806
Epoch 162/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.5607 - accuracy: 0.7692
Epoch 163/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.5515 - accuracy: 0.7688
Epoch 164/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.5072 - accuracy: 0.7813
Epoch 165/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.4753 - accuracy: 0.8003
Epoch 166/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.4832 - accuracy: 0.7899
Epoch 167/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.4578 - accuracy: 0.8057
Epoch 168/1500
153/153 [==============================] - 1s 6ms/step - loss: 0

153/153 [==============================] - 1s 7ms/step - loss: 0.3227 - accuracy: 0.8596
Epoch 316/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.3337 - accuracy: 0.8667
Epoch 317/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.3151 - accuracy: 0.8647
Epoch 318/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.3279 - accuracy: 0.8547
Epoch 319/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.3754 - accuracy: 0.8454
Epoch 320/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.3443 - accuracy: 0.8518
Epoch 321/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.3408 - accuracy: 0.8590
Epoch 322/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.3399 - accuracy: 0.8512
Epoch 323/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.3413 - accuracy: 0.8561
Epoch 324/1500
153/153 [==============================] - 1s 6ms/step - loss: 0

153/153 [==============================] - 1s 7ms/step - loss: 0.2527 - accuracy: 0.8848
Epoch 472/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2923 - accuracy: 0.8772
Epoch 473/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.3151 - accuracy: 0.8696
Epoch 474/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.3279 - accuracy: 0.8649
Epoch 475/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.3119 - accuracy: 0.8680
Epoch 476/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2677 - accuracy: 0.8827
Epoch 477/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2568 - accuracy: 0.8887
Epoch 478/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2496 - accuracy: 0.8905
Epoch 479/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2498 - accuracy: 0.8913
Epoch 480/1500
153/153 [==============================] - 1s 7ms/step - loss: 0

153/153 [==============================] - 1s 6ms/step - loss: 0.2350 - accuracy: 0.8934
Epoch 628/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2393 - accuracy: 0.8932
Epoch 629/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2419 - accuracy: 0.8946
Epoch 630/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2519 - accuracy: 0.8879
Epoch 631/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2863 - accuracy: 0.8788
Epoch 632/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.3404 - accuracy: 0.8585
Epoch 633/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.3029 - accuracy: 0.8807
Epoch 634/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2522 - accuracy: 0.8866
Epoch 635/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2574 - accuracy: 0.8924
Epoch 636/1500
153/153 [==============================] - 1s 6ms/step - loss: 0

153/153 [==============================] - 1s 7ms/step - loss: 0.2226 - accuracy: 0.9012
Epoch 784/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2026 - accuracy: 0.9104
Epoch 785/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2123 - accuracy: 0.9016
Epoch 786/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2181 - accuracy: 0.9057
Epoch 787/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2248 - accuracy: 0.9012
Epoch 788/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2363 - accuracy: 0.8979
Epoch 789/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.3663 - accuracy: 0.8602
Epoch 790/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2344 - accuracy: 0.8963
Epoch 791/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2292 - accuracy: 0.9004
Epoch 792/1500
153/153 [==============================] - 1s 6ms/step - loss: 0

153/153 [==============================] - 1s 8ms/step - loss: 0.2037 - accuracy: 0.9100
Epoch 940/1500
153/153 [==============================] - 1s 8ms/step - loss: 0.2041 - accuracy: 0.9108
Epoch 941/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2041 - accuracy: 0.9131
Epoch 942/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2227 - accuracy: 0.8971
Epoch 943/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2358 - accuracy: 0.8975
Epoch 944/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2402 - accuracy: 0.9008
Epoch 945/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.3272 - accuracy: 0.8797
Epoch 946/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2280 - accuracy: 0.9036
Epoch 947/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2247 - accuracy: 0.9053
Epoch 948/1500
153/153 [==============================] - 1s 6ms/step - loss: 0

153/153 [==============================] - 1s 7ms/step - loss: 0.2040 - accuracy: 0.9061
Epoch 1096/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.1965 - accuracy: 0.9100
Epoch 1097/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2233 - accuracy: 0.9006
Epoch 1098/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.1883 - accuracy: 0.9157
Epoch 1099/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.1863 - accuracy: 0.9139
Epoch 1100/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2045 - accuracy: 0.9094
Epoch 1101/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2080 - accuracy: 0.9100
Epoch 1102/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2140 - accuracy: 0.9039
Epoch 1103/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2301 - accuracy: 0.9069
Epoch 1104/1500
153/153 [==============================] - 1s 7ms/step 

153/153 [==============================] - 1s 6ms/step - loss: 0.1925 - accuracy: 0.9131
Epoch 1251/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.1799 - accuracy: 0.9147
Epoch 1252/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.1884 - accuracy: 0.9106
Epoch 1253/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.1838 - accuracy: 0.9159
Epoch 1254/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2264 - accuracy: 0.9014
Epoch 1255/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2211 - accuracy: 0.9041
Epoch 1256/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.1971 - accuracy: 0.9131
Epoch 1257/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2513 - accuracy: 0.9020
Epoch 1258/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2133 - accuracy: 0.9088
Epoch 1259/1500
153/153 [==============================] - 1s 6ms/step 

153/153 [==============================] - 1s 6ms/step - loss: 0.2870 - accuracy: 0.8875
Epoch 1406/1500
153/153 [==============================] - 1s 7ms/step - loss: 0.2966 - accuracy: 0.8848
Epoch 1407/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.2004 - accuracy: 0.9176
Epoch 1408/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.1597 - accuracy: 0.9252
Epoch 1409/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.1566 - accuracy: 0.9276
Epoch 1410/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.1624 - accuracy: 0.9229
Epoch 1411/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.1607 - accuracy: 0.9248
Epoch 1412/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.1680 - accuracy: 0.9203
Epoch 1413/1500
153/153 [==============================] - 1s 6ms/step - loss: 0.1822 - accuracy: 0.9143
Epoch 1414/1500
153/153 [==============================] - 1s 6ms/step 

In [61]:
y_predict = model.predict_classes(X_test)
print(y_predict)

[3 1 2 ... 2 3 0]


In [67]:
yTruePred=np.empty(len(y_predict), dtype=int)
for i in range(len(y_predict)):
    if y_predict[i]==y_test[i]:
      yTruePred[i] = y_predict[i]

In [69]:
print(len(y_test))
print(len(yTruePred))

2091
2091
